# 추천 시스템 실습

## 데이터셋 준비

### Surprise 라이브러리 소개

Surprise는 다양한 추천 알고리즘과 유사도 지표를 제공하는 Python SciKit입니다. 이를 통해 추천 시스템을 쉽게 구축하고 분석할 수 있습니다. 주요 기능은 데이터 로딩, 알고리즘 적용, 결과 평가 등을 포함합니다.


In [11]:
%%capture

# 필요한 패키지 설치 
%pip install aibasics

> **Note**: 구글 Colab에서는 위의 패키지 설치 명령어를 실행한 후에 세션을 재시작해야 합니다. 메뉴에서 **런타임** > **런타임 다시 시작**을 선택하세요.

### 데이터셋 로딩

Surprise 라이브러리를 사용하여 데이터셋을 로딩하는 방법에는 여러 가지가 있습니다. 파일에서 로딩, Pandas dataframe에서 로딩, 또는 Surprise가 제공하는 내장 데이터셋을 사용하는 방법 등이 있습니다.

#### 내장 데이터셋 로딩


In [1]:
from surprise import Dataset

# 내장 데이터셋 로딩
data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/yjlee/.surprise_data/ml-100k


#### 파일에서 데이터셋 로딩

```python
from surprise import Dataset
from surprise import Reader

# Reader 객체 생성
reader = Reader(line_format='user item rating', sep='\t', rating_scale=(1, 5))

# 파일에서 데이터셋 로딩
data = Dataset.load_from_file('파일 경로', reader=reader)
```

#### Pandas dataframe에서 데이터셋 로딩


In [2]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

# 데이터 딕셔너리 생성
ratings_dict = {
    "item": [1, 2, 3, 1, 2, 3, 2],
    "user": [1, 1, 1, 2, 2, 2, 3],
    "rating": [1, 2, 3, 1, 2, 3, 4],
}

# Pandas dataframe으로 변환
df = pd.DataFrame(ratings_dict)

# Reader 객체 생성
reader = Reader(rating_scale=(1, 5))

# 데이터셋 로딩
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)

### 데이터 전처리

데이터 전처리 과정에서는 결측치 처리, 데이터 정규화, 범주형 데이터의 수치화 등 데이터를 분석에 적합한 형태로 가공합니다. Surprise 라이브러리는 데이터 로딩 시 다양한 옵션을 제공하여 이러한 전처리 과정을 용이하게 합니다.

### 데이터셋 준비 중요성

추천 시스템 구축에서 데이터셋 준비는 매우 중요한 단계입니다. 데이터의 품질과 정제 정도는 추천 시스템의 성능에 직접적인 영향을 미치기 때문에, 데이터 로딩과 전처리 과정에 주의를 기울여야 합니다.

이 실습에서는 Surprise 라이브러리를 사용하여 데이터셋을 준비하는 방법을 배웠습니다. 이후 단계에서는 이 데이터를 사용하여 실제 추천 모델을 구축하고 평가하는 과정을 진행할 예정입니다.
